In [ ]:
import numpy as np
import pathlib
import sys

import mcpc16
import mcasm16

input_filename = r"programs\program.mcasm"
output_filename = r"output\program.mcbin"

input_filepath = pathlib.Path(input_filename)
output_filepath = pathlib.Path(output_filename)

print(f"Assembling '{input_filepath.resolve()}' -> '{output_filepath.resolve()}'")

## Load the instruction lines

### Include Paths

In [ ]:
include_paths: list[pathlib.Path] = [
    pathlib.Path.cwd() / "stdlib",
]

### Macros

Define additional macros

In [ ]:
default_macros: dict[str, str] = {
    "goto": "jump"
}

### Read the src lines

Load all lines from the specified input file

In [ ]:
# Read input lines
with open(input_filepath, "r") as input_file:
    src_lines = input_file.readlines()
src_lines = [line for line in src_lines]
print(f"Loaded {len(src_lines)} lines from source file:")

## Assemble the Program

In [ ]:
program = None
try:
    program = mcasm16.assemble(src_lines, str(input_filepath.absolute()), include_paths)
except mcasm16.AssemblySyntaxError as exception:
    print(exception, file=sys.stderr)
    exit(1)
except mcasm16.AssemblyIncludeError as exception:
    print(exception, file=sys.stderr)
    exit(1)

if program is None:
    print("Failed to assemble program.", file=sys.stderr)
    exit(1)

# print(f"Defined {len(program.labels)} labels:")
# print("\n".join([f"'{id}': instruction {(label.location):05d} in line {(program.instructions[label.location].source.text)}" for label in program.labels.values()]))
# print(f"\nAssembled {program.n_instructions} instructions:")
# print("\n".join([f"{i:05}: {line}" for i, line in enumerate(program.instructions)]))

### Output Mapping

In [ ]:
if program is not None:
    max_instruction_source_length = np.max([len(line.source.text) for line in program.instructions])
    for instruction in program.instructions:
        print(f"{instruction.source.text.ljust(max_instruction_source_length + 3)} # line {instruction.source.line + 1:5d} of unit \"{instruction.source.unit}\"")


## Write output file

Writes the final binary to the output file.

In [ ]:
if program is not None:
    with open(output_filepath, "wb") as output_file:
        program.binary.tofile(output_file, "")